# Vertex AI - Feature Store

**Prerequisites**
- `00 - Initial Setup`
- `01 - BigQuery - Data`
- `06 - Vertex AI - Training Pipeline and Serving`
    - for online prediction this used the endpoing defined in 06
    
**Resources**
- Based on:
    - https://cloud.google.com/vertex-ai/docs/featurestore/managing-featurestores
- API Documentation:
    - https://googleapis.dev/python/aiplatform/latest/aiplatform_v1beta1/services.html
    - https://googleapis.dev/python/aiplatform/latest/aiplatform_v1beta1/featurestore_service.html

**Overview**

<img src="architectures/statmike-mlops-07.png">


---
## Setup

Update Python library for aiplatform (Vertex AI)
- Restart the kernel: Menus > Kernel > Restart Kernel

In [9]:
!pip install --upgrade git+https://github.com/googleapis/python-aiplatform.git@main-test -q

Import Libraries

In [1]:
from google.cloud.aiplatform_v1beta1 import (FeaturestoreOnlineServingServiceClient, FeaturestoreServiceClient, types)
from google.protobuf.duration_pb2 import Duration
from google.protobuf.timestamp_pb2 import Timestamp
from google.protobuf.field_mask_pb2 import FieldMask

Setup GCP Parameters

In [2]:
# Locations
REGION = 'us-central1'
PROJECT_ID='statmike-mlops'
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

Clients for Feature Store:

In [3]:
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)
client_options = {"api_endpoint": API_ENDPOINT}
clients = {}

In [4]:
clients['fs'] = FeaturestoreServiceClient(client_options=client_options)

In [5]:
clients['fs_olserve'] = FeaturestoreOnlineServingServiceClient(client_options=client_options)

In [6]:
BASE_RESOURCE_PATH = clients['fs'].common_location_path(PROJECT_ID, REGION)

---
## Feature Store Data model
Feature Store organizes data with the following 3 important hierarchical concepts:

Featurestore -> EntityType -> Feature

- **Featurestore**: the place to store your features
    - **EntityType**: under a Featurestore, an EntityType describes an object to be modeled, real one or virtual one.
        - **Feature**: under an EntityType, a feature describes an attribute of the EntityType

For the digits data used in these examples, the feature store will be called digits_featurestore.  The store has 1 entity type: images.  The features will be the pixels and the target values.

---
## Create Feature Store

In [7]:
FEATURESTORE_ID = 'digits_featurestore'

In [8]:
featurestore_lro = clients['fs'].create_featurestore(
    types.featurestore_service.CreateFeaturestoreRequest(
        parent = BASE_RESOURCE_PATH,
        featurestore_id = FEATURESTORE_ID,
        featurestore=types.featurestore.Featurestore(
            display_name="Featurestore for handwritten digits",
            online_serving_config=types.featurestore.Featurestore.OnlineServingConfig(
                fixed_node_count=3
            ),
        ),
    )
)

In [9]:
featurestore_lro.result()

name: "projects/691911073727/locations/us-central1/featurestores/digits_featurestore"

Use `get_featurestore` to see details of specified feature store:

In [10]:
clients['fs'].get_featurestore(name=clients['fs'].featurestore_path(PROJECT_ID, REGION, FEATURESTORE_ID))

name: "projects/691911073727/locations/us-central1/featurestores/digits_featurestore"
create_time {
  seconds: 1628776342
  nanos: 168468000
}
update_time {
  seconds: 1628776342
  nanos: 231409000
}
etag: "AMEw9yNvwgSM34SQwu5Kjsl-Wa32RnTt2QRNwWt1Ow27hyRmGoaZCqBTcMQnoPX5UsU="
online_serving_config {
  fixed_node_count: 3
}
state: STABLE

Use `list_featurestores` to see details of all feature stores:

In [11]:
clients['fs'].list_featurestores(parent=PARENT)

ListFeaturestoresPager<featurestores {
  name: "projects/691911073727/locations/us-central1/featurestores/digits_featurestore"
  create_time {
    seconds: 1628776342
    nanos: 168468000
  }
  update_time {
    seconds: 1628776342
    nanos: 231409000
  }
  etag: "AMEw9yMvnUEelujdfpUe9a2_ZY3CrUDDwvz7TU3ncDxUTUHs7qmZwS7oNToy2Uyd5gM="
  online_serving_config {
    fixed_node_count: 3
  }
  state: STABLE
}
>

---
## Create Entity Type

In [12]:
ENTITYTYPE_ID = 'person'

In [13]:
entitytype_lro = clients['fs'].create_entity_type(
    types.featurestore_service.CreateEntityTypeRequest(
        parent=clients['fs'].featurestore_path(PROJECT_ID, REGION, FEATURESTORE_ID),
        entity_type_id = ENTITYTYPE_ID,
        entity_type=types.entity_type.EntityType(
            description="person entity",
            monitoring_config=types.featurestore_monitoring.FeaturestoreMonitoringConfig(
                snapshot_analysis=types.featurestore_monitoring.FeaturestoreMonitoringConfig.SnapshotAnalysis(
                    monitoring_interval=Duration(seconds=900),  # 15 minutes
                ),
            ),
        ),
    )
)

In [14]:
entitytype_lro.result()

name: "projects/691911073727/locations/us-central1/featurestores/digits_featurestore/entityTypes/person"
etag: "AMEw9yONRE-R4V6Vxt2F6Ugzv28w0Vc_IzjK7mu_iaCpSnu5TpPl"

Use `list_entity_types` to see details of all entity types:

In [15]:
clients['fs'].list_entity_types(parent=PARENT+'/featurestores/{}'.format(FEATURESTORE_ID))

ListEntityTypesPager<entity_types {
  name: "projects/691911073727/locations/us-central1/featurestores/digits_featurestore/entityTypes/person"
  description: "person entity"
  create_time {
    seconds: 1628776474
    nanos: 237856000
  }
  update_time {
    seconds: 1628776474
    nanos: 237856000
  }
  etag: "AMEw9yNDsDPyNTPT5z4CQ6rym3Bw7mjUkREnIZTiKpqwTkn3qJUows2SXkwhf7yMr6Y="
  monitoring_config {
    snapshot_analysis {
      monitoring_interval {
        seconds: 86400
      }
    }
  }
}
>

---
## Create Features

Get the schema of the data source for new features:

In [16]:
%%bigquery schema
SELECT * 
FROM `statmike-mlops.digits.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS`
WHERE table_name = 'digits_source'

Downloading: 100%|██████████| 66/66 [00:01<00:00, 58.14rows/s]


In [17]:
schema

,table_catalog,table_schema,table_name,column_name,field_path,data_type,description
0,statmike-mlops,digits,digits_source,p0,p0,FLOAT64,None
1,statmike-mlops,digits,digits_source,p1,p1,FLOAT64,None
2,statmike-mlops,digits,digits_source,p2,p2,FLOAT64,None
3,statmike-mlops,digits,digits_source,p3,p3,FLOAT64,None
4,statmike-mlops,digits,digits_source,p4,p4,FLOAT64,None
...,...,...,...,...,...,...,...
61,statmike-mlops,digits,digits_source,p61,p61,FLOAT64,None
62,statmike-mlops,digits,digits_source,p62,p62,FLOAT64,None
63,statmike-mlops,digits,digits_source,p63,p63,FLOAT64,None
64,statmike-mlops,digits,digits_source,target,target,INT64,None


Prepare a request for `batch_create_features`:
- specification for the features, data type and descriptions ....

In [18]:
REQUESTS = []
for i in range(schema.shape[0]):
    
    if schema['data_type'][i] == 'STRING': value_type = types.feature.Feature.ValueType.STRING
    elif schema['data_type'][i] == 'INT64': value_type = types.feature.Feature.ValueType.INT64
    elif schema['data_type'][i] == 'FLOAT64': value_type = types.feature.Feature.ValueType.DOUBLE
    
    if schema['description'][i] == None: description = schema['column_name'][i]
    else: description = schema['description'][i]
    
    REQUESTS.append(
        types.featurestore_service.CreateFeatureRequest(
            feature=types.feature.Feature(
                value_type = value_type,
                description = description,
                # optional, monitoring_config here as override, otherwise it inherits from entity_type
            ),
            feature_id = schema['column_name'][i].lower(),
        )    
    )

In [19]:
batchfeatures = clients['fs'].batch_create_features(
    parent = clients['fs'].entity_type_path(PROJECT_ID, REGION, FEATURESTORE_ID, ENTITYTYPE_ID),
    requests = REQUESTS,
)

In [20]:
list(item.name for item in batchfeatures.result().features)

['projects/691911073727/locations/us-central1/featurestores/digits_featurestore/entityTypes/person/features/p0',
 'projects/691911073727/locations/us-central1/featurestores/digits_featurestore/entityTypes/person/features/p1',
 'projects/691911073727/locations/us-central1/featurestores/digits_featurestore/entityTypes/person/features/p2',
 'projects/691911073727/locations/us-central1/featurestores/digits_featurestore/entityTypes/person/features/p3',
 'projects/691911073727/locations/us-central1/featurestores/digits_featurestore/entityTypes/person/features/p4',
 'projects/691911073727/locations/us-central1/featurestores/digits_featurestore/entityTypes/person/features/p5',
 'projects/691911073727/locations/us-central1/featurestores/digits_featurestore/entityTypes/person/features/p6',
 'projects/691911073727/locations/us-central1/featurestores/digits_featurestore/entityTypes/person/features/p7',
 'projects/691911073727/locations/us-central1/featurestores/digits_featurestore/entityTypes/pers

---
## Search Features
Search goes across all Feature Stores and Entity Types.

Also, use the list_features function to list all.

In [21]:
# return the first feature:
list(clients['fs'].search_features(location=BASE_RESOURCE_PATH))[0]

name: "projects/691911073727/locations/us-central1/featurestores/digits_featurestore/entityTypes/person/features/p0"
description: "p0"
create_time {
  seconds: 1628776601
  nanos: 645000
}
update_time {
  seconds: 1628776601
  nanos: 645000
}

In [22]:
# find all features with INT64 value type
list(clients['fs'].search_features(types.featurestore_service.SearchFeaturesRequest(location=BASE_RESOURCE_PATH, query="value_type=INT64")))

[name: "projects/691911073727/locations/us-central1/featurestores/digits_featurestore/entityTypes/person/features/target"
 description: "target"
 create_time {
   seconds: 1628776601
   nanos: 90387000
 }
 update_time {
   seconds: 1628776601
   nanos: 90387000
 }]

In [23]:
# find all features of the form p6* with DOUBLE value type
list(clients['fs'].search_features(types.featurestore_service.SearchFeaturesRequest(location=BASE_RESOURCE_PATH, query="feature_id:p6* AND value_type=DOUBLE")))

[name: "projects/691911073727/locations/us-central1/featurestores/digits_featurestore/entityTypes/person/features/p6"
 description: "p6"
 create_time {
   seconds: 1628776601
   nanos: 9434000
 }
 update_time {
   seconds: 1628776601
   nanos: 9434000
 },
 name: "projects/691911073727/locations/us-central1/featurestores/digits_featurestore/entityTypes/person/features/p60"
 description: "p60"
 create_time {
   seconds: 1628776601
   nanos: 84773000
 }
 update_time {
   seconds: 1628776601
   nanos: 84773000
 },
 name: "projects/691911073727/locations/us-central1/featurestores/digits_featurestore/entityTypes/person/features/p61"
 description: "p61"
 create_time {
   seconds: 1628776601
   nanos: 85808000
 }
 update_time {
   seconds: 1628776601
   nanos: 85808000
 },
 name: "projects/691911073727/locations/us-central1/featurestores/digits_featurestore/entityTypes/person/features/p62"
 description: "p62"
 create_time {
   seconds: 1628776601
   nanos: 87419000
 }
 update_time {
   seconds

---
## Import Feature Values
- BigQuery (THIS DEMO)
- Avro
- CSV

Prepare a source table with timestamp (update_time) and unique id's (image_id) for each entity

In [26]:
%%bigquery
CREATE OR REPLACE TABLE `statmike-mlops.digits.digits_featurestore_import` AS
WITH A AS (SELECT GENERATE_UUID() unique_id, target_OE as target_oe, CAST(FLOOR(10*RAND()) AS INT64) daytrick, * EXCEPT(target_OE) FROM `statmike-mlops.digits.digits_source`)
SELECT * EXCEPT(daytrick), DATE_SUB(CURRENT_TIMESTAMP, INTERVAL daytrick DAY) AS update_time FROM A

Query complete after 0.00s: 100%|██████████| 3/3 [00:00<00:00, 2018.76query/s]                        


""


Create Feature specification for each feature in the input source:

In [27]:
FEATURE_SPECS = []
for i in range(schema.shape[0]):
    FEATURE_SPECS.append(types.featurestore_service.ImportFeatureValuesRequest.FeatureSpec(id=schema['column_name'][i].lower()))

In [28]:
import_request = types.featurestore_service.ImportFeatureValuesRequest(
    entity_type = clients['fs'].entity_type_path(PROJECT_ID, REGION, FEATURESTORE_ID, ENTITYTYPE_ID),
    bigquery_source = types.BigQuerySource(input_uri='bq://statmike-mlops.digits.digits_featurestore_import'),
    feature_time_field = "update_time",
    feature_time = Timestamp().GetCurrentTime(),
    entity_id_field = "unique_id",
    feature_specs = FEATURE_SPECS,
    worker_count = 4,
)

In [29]:
importjob = clients['fs'].import_feature_values(import_request)

In [30]:
importjob.result()

imported_entity_count: 1797
imported_feature_value_count: 118602

---
## Serving Features

Retrieve a list of entity id's:

In [33]:
%%bigquery unique_id
SELECT unique_id FROM `statmike-mlops.digits.digits_featurestore_import`

Downloading: 100%|██████████| 1797/1797 [00:00<00:00, 2011.51rows/s]


In [34]:
id_values = list(unique_id['unique_id'])
id_values[0:5]

['9b60e611-eca0-4dc7-a632-4c717206f1d2',
 'e8c2a463-047f-4b05-8055-8ae2e6aaed16',
 'ca5f4898-3eea-4753-8d0d-4c07be8478df',
 '566a363c-ef06-46fa-afdc-5a1c402d1924',
 '78ca1f05-09dc-41eb-9611-971440c25b3b']

### Online - One (Specific Date)

In [ ]:
single = clients['fs_olserve'].read_feature_values(
    types.featurestore_online_service.ReadFeatureValuesRequest(
        entity_type = clients['fs'].entity_type_path(PROJECT_ID, REGION, FEATURESTORE_ID, ENTITYTYPE_ID),
        entity_id=id_values[0],
        feature_selector = types.FeatureSelector(id_matcher=types.IdMatcher(ids=['*'])),
    )
)

### Online - One (Current Value)

In [35]:
single = clients['fs_olserve'].read_feature_values(
    types.featurestore_online_service.ReadFeatureValuesRequest(
        entity_type = clients['fs'].entity_type_path(PROJECT_ID, REGION, FEATURESTORE_ID, ENTITYTYPE_ID),
        entity_id=id_values[0],
        feature_selector = types.FeatureSelector(id_matcher=types.IdMatcher(ids=['*'])),
    )
)

In [36]:
print(list(item.id for item in single.header.feature_descriptors))

['p31', 'p36', 'p29', 'p18', 'p16', 'p63', 'p27', 'p5', 'p61', 'p38', 'p19', 'p55', 'p12', 'p41', 'p6', 'p7', 'p14', 'p56', 'p54', 'p22', 'p60', 'p43', 'p42', 'p51', 'p39', 'p53', 'p20', 'p0', 'p47', 'p52', 'p34', 'p8', 'p32', 'p2', 'p28', 'p37', 'p58', 'p10', 'p3', 'p45', 'p25', 'p44', 'p50', 'p49', 'p13', 'p62', 'p33', 'p40', 'p1', 'p46', 'p24', 'p57', 'target_oe', 'p59', 'p21', 'p48', 'p9', 'p17', 'p23', 'p15', 'p35', 'p4', 'target', 'p11', 'p30', 'p26']


In [37]:
print(list(item.value.double_value for item in single.entity_view.data))

[0.0, 0.0, 10.0, 12.0, 0.0, 0.0, 0.0, 3.0, 4.0, 7.0, 8.0, 0.0, 12.0, 2.0, 0.0, 0.0, 2.0, 0.0, 1.0, 4.0, 14.0, 4.0, 16.0, 14.0, 0.0, 15.0, 0.0, 0.0, 0.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 13.0, 1.0, 8.0, 9.0, 16.0, 3.0, 7.0, 14.0, 0.0, 15.0, 0.0, 8.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 12.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 0.0, 15.0, 7.0, 13.0]


#### Prediction

In [38]:
from google.cloud.aiplatform_v1beta1 import (PredictionServiceClient, EndpointServiceClient)
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value

Function to retrieve online features for an entity and prepare them for prediction:

In [39]:
def fetch_single(entity_id):
    single = clients['fs_olserve'].read_feature_values(
        types.featurestore_online_service.ReadFeatureValuesRequest(
            entity_type = clients['fs'].entity_type_path(PROJECT_ID, REGION, FEATURESTORE_ID, ENTITYTYPE_ID),
            entity_id=entity_id,
            feature_selector = types.FeatureSelector(id_matcher=types.IdMatcher(ids=['*'])),
        )
    )
    
    newob = {}
    features = list(item.id for item in single.header.feature_descriptors)
    for e, f in enumerate(features):
        if f.startswith('p'):
            newob[f] = single.entity_view.data[e].value.double_value
    
    return newob

In [40]:
newob = fetch_single(id_values[0])

Pick an endpoint from those setup in this project (all have the same model parameters):

In [41]:
clients['endpoint'] = EndpointServiceClient(client_options=client_options)

In [42]:
list((ep.name, ep.display_name) for ep in clients['endpoint'].list_endpoints(parent=PARENT))

[('projects/691911073727/locations/us-central1/endpoints/87943338036035584',
  'ENDPOINT_06_AIP_DIGITS_20210812121445'),
 ('projects/691911073727/locations/us-central1/endpoints/2317225153584431104',
  'ENDPOINT_05_AIP_DIGITS_20210812120620'),
 ('projects/691911073727/locations/us-central1/endpoints/2069527174079053824',
  'ENDPOINT_04_KERAS-DIGITS'),
 ('projects/691911073727/locations/us-central1/endpoints/9095142592777027584',
  'ENDPOINT_03_BQML-DIGITS')]

In [43]:
ep = clients['endpoint'].list_endpoints(parent=PARENT).endpoints[1].name
ep

'projects/691911073727/locations/us-central1/endpoints/2317225153584431104'

Python: Request a prediction for the entity's features:

In [44]:
clients['prediction'] = PredictionServiceClient(client_options=client_options)

In [45]:
response = clients['prediction'].predict(endpoint=ep, instances=[json_format.ParseDict(newob, Value())], parameters=json_format.ParseDict({}, Value()))

In [46]:
response

predictions {
  list_value {
    values {
      number_value: 0.999998927
    }
    values {
      number_value: 6.41524958e-11
    }
    values {
      number_value: 2.10591722e-09
    }
    values {
      number_value: 3.92068626e-12
    }
    values {
      number_value: 7.72907129e-07
    }
    values {
      number_value: 1.84147461e-10
    }
    values {
      number_value: 7.12186177e-09
    }
    values {
      number_value: 2.12504569e-10
    }
    values {
      number_value: 3.02985853e-07
    }
    values {
      number_value: 2.69612249e-11
    }
  }
}
deployed_model_id: "2519491312630104064"

REST: Request a prediction for the entity's features:

In [47]:
import json
with open('request.json','w') as file:
    file.write(json.dumps({"instances": [newob]}))

In [48]:
!curl -X POST \
-H "Authorization: Bearer "$(gcloud auth application-default print-access-token) \
-H "Content-Type: application/json; charset=utf-8" \
-d @request.json \
https://{API_ENDPOINT}/v1/{ep}:predict

{
  "predictions": [
    [
      0.999998927,
      6.41524958e-11,
      2.10591722e-09,
      3.92068626e-12,
      7.72907129e-07,
      1.84147461e-10,
      7.12186177e-09,
      2.12504569e-10,
      3.02985853e-07,
      2.69612249e-11
    ]
  ],
  "deployedModelId": "2519491312630104064"
}


CLI: Request a prediction for the entity's features:

In [49]:
!gcloud beta ai endpoints predict {ep.rsplit('/',1)[-1]} --region={REGION} --json-request=request.json

Using endpoint [https://us-central1-prediction-aiplatform.googleapis.com/]
[[0.999998927, 6.41524958e-11, 2.10591722e-09, 3.92068626e-12, 7.72907129e-07, 1.84147461e-10, 7.12186177e-09, 2.12504569e-10, 3.02985853e-07, 2.69612249e-11]]


### Online - Multi

In [50]:
multi = clients['fs_olserve'].streaming_read_feature_values(
    types.featurestore_online_service.StreamingReadFeatureValuesRequest(
        entity_type = clients['fs'].entity_type_path(PROJECT_ID, REGION, FEATURESTORE_ID, ENTITYTYPE_ID),
        entity_ids = id_values[0:3],
        feature_selector = types.FeatureSelector(id_matcher=types.IdMatcher(ids=['*'])),
    )
)

In [51]:
for i in multi:
    print(i.entity_view.entity_id)
    print(list(item.value.double_value for item in i.entity_view.data))


[]
9b60e611-eca0-4dc7-a632-4c717206f1d2
[0.0, 0.0, 0.0, 1.0, 15.0, 4.0, 2.0, 9.0, 1.0, 14.0, 0.0, 13.0, 0.0, 0.0, 3.0, 15.0, 10.0, 0.0, 16.0, 0.0, 0.0, 7.0, 0.0, 0.0, 4.0, 8.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 0.0, 9.0, 0.0, 14.0, 13.0, 0.0, 15.0, 3.0, 8.0, 0.0, 0.0, 7.0, 0.0, 0.0, 15.0, 5.0, 0.0, 16.0, 12.0, 0.0, 2.0, 12.0, 0.0, 13.0, 8.0, 0.0, 0.0, 0.0, 0.0, 16.0, 0.0, 7.0, 4.0]
ca5f4898-3eea-4753-8d0d-4c07be8478df
[0.0, 0.0, 0.0, 3.0, 13.0, 6.0, 2.0, 9.0, 1.0, 11.0, 0.0, 9.0, 0.0, 0.0, 4.0, 13.0, 6.0, 0.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 10.0, 4.0, 0.0, 0.0, 0.0, 0.0, 2.0, 6.0, 0.0, 8.0, 0.0, 16.0, 8.0, 0.0, 13.0, 6.0, 5.0, 0.0, 0.0, 7.0, 0.0, 0.0, 10.0, 5.0, 0.0, 11.0, 14.0, 0.0, 2.0, 11.0, 0.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 8.0, 12.0]
e8c2a463-047f-4b05-8055-8ae2e6aaed16
[0.0, 0.0, 0.0, 1.0, 16.0, 1.0, 0.0, 10.0, 2.0, 16.0, 0.0, 16.0, 0.0, 0.0, 5.0, 14.0, 14.0, 6.0, 16.0, 0.0, 0.0, 5.0, 0.0, 0.0, 1.0, 8.0, 14.0, 0.0, 0.0, 0.0, 0.0, 3.0, 10.0, 0.0, 16.0, 0.0, 13.

### Batch (For training or large scale prediction)

In [52]:
from google.cloud import bigquery

DESTINATION_DATASET = 'digits_training'

clients['bq'] = bigquery.Client()
dataset_id = "{}.{}".format(clients['bq'].project, DESTINATION_DATASET)
dataset = bigquery.Dataset(dataset_id)
dataset.location = REGION
dataset = clients['bq'].create_dataset(dataset, exists_ok = True)

In [53]:
batch_request = types.featurestore_service.ExportFeatureValuesRequest(
    entity_type = clients['fs'].entity_type_path(PROJECT_ID, REGION, FEATURESTORE_ID, ENTITYTYPE_ID),
    snapshot_export = types.ExportFeatureValuesRequest.SnapshotExport(snapshot_time = Timestamp().GetCurrentTime()),
    destination = types.FeatureValueDestination(bigquery_destination = types.BigQueryDestination(output_uri='bq://statmike-mlops.digits_training.training')),
    feature_selector = types.FeatureSelector(id_matcher=types.IdMatcher(ids=['*']))
)

In [54]:
batchjob = clients['fs'].export_feature_values(batch_request)

In [55]:
batchjob.result()

---
## Remove Resources
see notebook "XX - Cleanup"